In [20]:
import numpy as np
import matplotlib.pyplot as plt

# Define problem parameters
NUM_CUSTOMERS = 10
NUM_VEHICLES = 3
CAPACITY = 20
DE_POPULATION_SIZE = 50
DE_MAX_GENERATIONS = 50
DE_CR = 0.8
DE_F = 0.5

# Generate random customer demands
#np.random.seed(42)
customer_demands = np.random.randint(1, 3, size=NUM_CUSTOMERS)

# Define distance matrix
locations = np.random.rand(NUM_CUSTOMERS + 1, 2)
distances = np.zeros((NUM_CUSTOMERS + 1, NUM_CUSTOMERS + 1))
for i in range(NUM_CUSTOMERS + 1):
    for j in range(i + 1, NUM_CUSTOMERS + 1):
        distances[i, j] = np.linalg.norm(locations[i] - locations[j])
        distances[j, i] = distances[i, j]

# Define fitness function
def fitness(solution):
    """
    Computes the total distance traveled by all vehicles in the solution, given a list of routes.
    """
    routes = np.split(solution, np.where(solution == 0)[0])
    routes = [r for r in routes if len(r) > 0]
    total_distance = 0
    for r in routes:
        route_distance = 0
        for i in range(len(r) - 1):
            route_distance += distances[r[i], r[i + 1]]
        total_distance += route_distance
    return total_distance

# Define Differential Evolution algorithm
def differential_evolution(fitness_fn, population_size, max_generations, crossover_rate, f):
    """
    Implements the Differential Evolution algorithm for optimizing a given fitness function.
    """
    population = np.random.randint(NUM_CUSTOMERS + 1, size=(population_size, NUM_CUSTOMERS + NUM_VEHICLES))
    best_fitness = np.inf
    best_solution = None
    fitness_history = []
    for i in range(max_generations):
        #population_fitness = []
        #for k in range(population_size):
        #    population_fitness.append(fitness_fn(population[k]))
        #print(population_fitness)
        for j in range(population_size):
            # Select three random individuals from the population
            a, b, c = population[np.random.choice(population_size, size=3, replace=False)]
            # Perform crossover operation
            mask = np.random.rand(NUM_CUSTOMERS + NUM_VEHICLES) < crossover_rate
            trial_solution = np.where(mask, a + f * (b - c), population[j])
            # Ensure that the trial solution is valid
            trial_solution = np.unique(np.clip(trial_solution, 0, NUM_CUSTOMERS)).astype(int)
            trial_solution = trial_solution[trial_solution < NUM_CUSTOMERS]
            print(np.sum(customer_demands[trial_solution]))
            if np.sum(customer_demands[trial_solution]) > CAPACITY:
                trial_solution = population[j]
            # Evaluate fitness of trial solution
            trial_fitness = fitness_fn(trial_solution)
            # Update population with trial solution if it is better
            #print('Trial Fitness: ' + str(trial_fitness) + ', Population Fitness: '  + str(fitness_fn(population[j])))
            if trial_fitness < fitness_fn(population[j]):
                print(population[j])
                population[j] = trial_solution
                print(population[j])
                if trial_fitness < best_fitness:
                    best_fitness = trial_fitness
                    best_solution = trial_solution
        fitness_history.append(best_fitness)
        print(f"Generation {i + 1}/{max_generations}: Best fitness = {best_fitness}")
    return best_solution, best_fitness, fitness_history

# Run Differential Evolution algorithm
best_solution, best_fitness, fitness_history = differential_evolution(
    fitness_fn=fitness,
    population_size=DE_POPULATION_SIZE,
    max_generations=DE_MAX_GENERATIONS,
    crossover_rate=DE_CR,
    f=DE_F
)
    
# Print best solution and fitness
print(f"Best solution: {best_solution}")
print(f"Best fitness: {best_fitness}")

# Visualize fitness history
plt.plot(fitness_history)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Fitness History")
plt.show()

# Visualize solution
routes = np.split(best_solution, np.where(best_solution == 0)[0])
routes = [r for r in routes if len(r) > 0]
colors = ["r", "g", "b", "c", "m", "y", "k"]
fig, ax = plt.subplots()
for i, r in enumerate(routes):
    x = locations[r, 0]
    y = locations[r, 1]
    ax.scatter(x[1:-1], y[1:-1], color=colors[i % len(colors)])
    ax.plot(x, y, color=colors[i % len(colors)])
ax.scatter(locations[0, 0], locations[0, 1], marker="s", color="k")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_title("Best Solution")
plt.show()


15
[ 4  9  5  4  9  7  5  4  5  9  4 10  1]


ValueError: could not broadcast input array from shape (9,) into shape (13,)